In [1]:
import torch 
import pickle
from tqdm import tqdm
from transformers import BertTokenizer,BertModel
from datasets import load_dataset
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, random_split, DataLoader
from transformers import BertTokenizer,BertModel
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import balanced_accuracy_score
import pandas as pd

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = 'cpu'
print(device)

/home/prtissot/Documents/IA/ENV/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda:0


In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') 
# model = BertModel.from_pretrained("bert-base-uncased")

In [3]:
df = pd.read_csv('data_reviews/Atlanta_reviews.csv')
# with open("data_reviews/Atlanta_trainValidTest/labels.pickle", "rb") as f:
#     x = pickle.load(f)
# df_count = df.groupby('business_id').count()
# df_count = df_count[df_count.index.isin(x)]
# df = df[df.business_id.isin(df_count.index)]

/tmp/ipykernel_36167/85183323.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data_reviews/Atlanta_reviews.csv')


In [4]:
def get_sup(df, nb=100):
    df_count = df.groupby('business_id').count()
    df_plus = df_count[df_count.review_id >= nb]
    return df[df.business_id.isin(df_plus.index)]

def get_groupby_business(df):
    return df.groupby('business_id').count()

def get_groupby_price(df):
    return df.groupby('price').count()

In [5]:
df = get_sup(df,2)
df.price = df.price.astype(float).fillna(0.0)
get_groupby_price(df)

,business_id,review_id,user id,stars,review_date,text,name,categories,address,postal code,text_length
price,,,,,,,,,,,
0.0,93758,93758,93758,93758,93758,93758,13,13,13,13,13
1.0,99670,99670,99670,99670,99670,99670,99670,99670,99670,99670,99670
2.0,437486,437486,437486,437486,437486,437486,437486,437486,437486,437486,437486
3.0,44621,44621,44621,44621,44621,44621,44621,44621,44621,44621,44621
4.0,5754,5754,5754,5754,5754,5754,5754,5754,5754,5754,5754


In [6]:
df['review_date'] = pd.to_datetime(df['review_date'])  
df = df.loc[(df['review_date'] >= '2008-01-01') & (df['review_date'] <= '2020-01-01')]
print(df.shape)
get_groupby_price(df)

(540160, 12)


,business_id,review_id,user id,stars,review_date,text,name,categories,address,postal code,text_length
price,,,,,,,,,,,
0.0,13,13,13,13,13,13,13,13,13,13,13
1.0,91799,91799,91799,91799,91799,91799,91799,91799,91799,91799,91799
2.0,401769,401769,401769,401769,401769,401769,401769,401769,401769,401769,401769
3.0,41307,41307,41307,41307,41307,41307,41307,41307,41307,41307,41307
4.0,5272,5272,5272,5272,5272,5272,5272,5272,5272,5272,5272


In [7]:
get_groupby_business(df)

,review_id,user id,stars,review_date,text,name,categories,address,postal code,price,text_length
business_id,,,,,,,,,,,
-0JXXCMUpe29dQ-f6pG9OA,22,22,22,22,22,22,22,22,22,22,22
-0OWS89ebRdvzOQkjptyEw,31,31,31,31,31,31,31,31,31,31,31
-4P7VlKa9n-TwOP2_nWCtw,80,80,80,80,80,80,80,80,80,80,80
-5VyAi8GR34xmDAgFZTitg,425,425,425,425,425,425,425,425,425,425,425
-6VuqBXDDJXv8CRm_CM2qg,28,28,28,28,28,28,28,28,28,28,28
...,...,...,...,...,...,...,...,...,...,...,...
zy75CWA_oJYEhtXz8OOQsg,68,68,68,68,68,68,68,68,68,68,68
zyYNLYUVabUXkwq___Q3ug,27,27,27,27,27,27,27,27,27,27,27
zylYdVSB_KCGsB-Vunncsw,30,30,30,30,30,30,30,30,30,30,30


In [8]:
df = get_sup(df,100)
print(df.shape)

(462381, 12)


In [9]:
get_groupby_price(df)

,business_id,review_id,user id,stars,review_date,text,name,categories,address,postal code,text_length
price,,,,,,,,,,,
1.0,55375,55375,55375,55375,55375,55375,55375,55375,55375,55375,55375
2.0,364563,364563,364563,364563,364563,364563,364563,364563,364563,364563,364563
3.0,37669,37669,37669,37669,37669,37669,37669,37669,37669,37669,37669
4.0,4774,4774,4774,4774,4774,4774,4774,4774,4774,4774,4774


In [10]:
get_groupby_business(df)

,review_id,user id,stars,review_date,text,name,categories,address,postal code,price,text_length
business_id,,,,,,,,,,,
-5VyAi8GR34xmDAgFZTitg,425,425,425,425,425,425,425,425,425,425,425
-AIX1rem_OF-9Et3p_K9Gg,310,310,310,310,310,310,310,310,310,310,310
-CHazLwo2j2G8gWEZN53hA,755,755,755,755,755,755,755,755,755,755,755
-DISJqPp4zcDVw7R-MOjog,186,186,186,186,186,186,186,186,186,186,186
-EzfZm6rTohZdD9tfQaMyA,156,156,156,156,156,156,156,156,156,156,156
...,...,...,...,...,...,...,...,...,...,...,...
zpKvO2SOXHV9cxvBm6q4Fg,203,203,203,203,203,203,203,203,203,203,203
zpOcXfa6bbW6AG5L60UL_A,202,202,202,202,202,202,202,202,202,202,202
ztY4uPNUTWMN9LT3L5mD3Q,761,761,761,761,761,761,761,761,761,761,761


## Resampling

In [11]:
# df_p = df.groupby('business_id')
# df = df_p.apply(lambda x: x.sample(df_p.size().min(), replace=True, random_state=1).reset_index(drop=True)).reset_index(drop=True)

## Weight loss

In [12]:
nb_resto = get_groupby_price(df)['business_id'].tolist()

In [13]:
MAX_NB_RESTO = max(nb_resto)

In [15]:
ratio_resto = [MAX_NB_RESTO/p for p in nb_resto]
print(ratio_resto)
WEIGHT = torch.FloatTensor(ratio_resto).to(device)

[6.583530474040632, 1.0, 9.67806419071385, 76.36426476749057]


In [16]:
get_groupby_price(df)

,business_id,review_id,user id,stars,review_date,text,name,categories,address,postal code,text_length
price,,,,,,,,,,,
1.0,55375,55375,55375,55375,55375,55375,55375,55375,55375,55375,55375
2.0,364563,364563,364563,364563,364563,364563,364563,364563,364563,364563,364563
3.0,37669,37669,37669,37669,37669,37669,37669,37669,37669,37669,37669
4.0,4774,4774,4774,4774,4774,4774,4774,4774,4774,4774,4774


In [17]:
get_groupby_business(df)

,review_id,user id,stars,review_date,text,name,categories,address,postal code,price,text_length
business_id,,,,,,,,,,,
-5VyAi8GR34xmDAgFZTitg,425,425,425,425,425,425,425,425,425,425,425
-AIX1rem_OF-9Et3p_K9Gg,310,310,310,310,310,310,310,310,310,310,310
-CHazLwo2j2G8gWEZN53hA,755,755,755,755,755,755,755,755,755,755,755
-DISJqPp4zcDVw7R-MOjog,186,186,186,186,186,186,186,186,186,186,186
-EzfZm6rTohZdD9tfQaMyA,156,156,156,156,156,156,156,156,156,156,156
...,...,...,...,...,...,...,...,...,...,...,...
zpKvO2SOXHV9cxvBm6q4Fg,203,203,203,203,203,203,203,203,203,203,203
zpOcXfa6bbW6AG5L60UL_A,202,202,202,202,202,202,202,202,202,202,202
ztY4uPNUTWMN9LT3L5mD3Q,761,761,761,761,761,761,761,761,761,761,761


In [18]:
# df['weight'] = df.price.apply(lambda x: ratio_price[int(x)-1])

In [19]:
# df[['price', 'weight']].head(15)

In [20]:
# df['review_date'] = pd.to_datetime(df['review_date'])  
# df = df.loc[(df['review_date'] > '2008-01-01') & (df['review_date'] <= '2020-01-01')]

In [21]:
# display(df)
# print(df.columns)

In [22]:
# def get_sup(df, nb=100):
#     df_count = df.groupby('business_id').count()
#     df_plus = df_count[df_count.review_id >= nb]
#     return df[df.business_id.isin(df_plus.index)]

# df = get_sup(df,2)
# df = df.loc[(df['review_date'] > '2008-01-01') & (df['review_date'] <= '2020-01-01')]
# df = get_sup(df)

In [23]:
# df

In [24]:
labels = df['business_id']
print(len(labels))
LABELS = list(labels.unique())
# print(LABELS)
NB_CLASSES = len(LABELS)
print(NB_CLASSES)

462381
1404


In [25]:
# sent = df['text']
# sent = sent.unique()
# print(len(sent))
# sentToken = [tokenizer.tokenize(str(s)) for s in sent]
# NB_TOKEN = max(len(s) for s in sentToken)+2
# print(NB_TOKEN)

In [26]:
# sentTest = max(sentToken, key = len)
# print(sentTest)
# tok = tokenizer(sentTest, add_special_tokens=True, padding=True, return_tensors="pt")
# print(tok['input_ids'])
# print(tok['input_ids'].shape)

In [27]:
NB_TOKEN = 512

In [28]:
def split(df,ratio = 0.9):
    msk = np.random.rand(len(df)) < ratio
    train = df[msk]
    test = df[~msk]
    return train, test

In [29]:
train, test = split(df)
train, val = split(train)

In [30]:
print(len(train))
print(len(val))
print(len(test))

374391
41755
46235


In [31]:
class MyDataset(Dataset):
    def __init__(self, sentence, labels):
        self.wrapped_input = tokenizer(sentence, max_length=NB_TOKEN, add_special_tokens=True, truncation=True,
                          padding='max_length', return_tensors="pt")
        lab_idx = np.array([LABELS.index(l) for l in labels])
        self.labels = np.zeros((lab_idx.size, NB_CLASSES))
        self.labels[np.arange(lab_idx.size), lab_idx] = 1
        # self.labels = np.array(labels).astype(float)
    def __getitem__(self, idx):
        input_dict = {}
        for k in self.wrapped_input.keys():
            input_dict[k] = self.wrapped_input[k][idx]
        return input_dict, self.labels[idx]
    
    def __len__(self):
        return len(self.labels)

In [ ]:
BATCH_SIZE = 100

trainset = MyDataset(train['text'].astype(str).tolist(), train['business_id'])
print('train done')
valset = MyDataset(val['text'].astype(str).tolist(), val['business_id'])

train_loader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(valset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
# with open('train_dataloader.pkl', 'wb') as f:
#     pickle.dump(train_loader, f)
# with open('val_dataloader.pkl', 'wb') as f:
#     pickle.dump(val_loader, f)

In [ ]:
# with open('train_dataloader.pkl', 'rb') as f:
#     train_loader = pickle.load(f)
# with open('val_dataloader.pkl', 'rb') as f:
#     val_loader = pickle.load(f)

In [ ]:
# df['text'][:10].astype(str).tolist()

In [ ]:
class BERT_classifier(nn.Module):
    def __init__(self, num_label):
        super().__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        for param in self.bert.parameters():
            param.requires_grad = False
        self.classifier = nn.Sequential(
            nn.Linear(self.bert.config.hidden_size, 1024),
            nn.ReLU(),
            nn.Linear(1024, num_label),
            # nn.ReLU(),
            # nn.Linear(1500, 2000),
            # nn.ReLU(),
            # nn.Linear(2000, 2500),
            # nn.ReLU(),
            # nn.Linear(2500, num_label),
            # nn.ReLU(),
            # nn.Linear(3000, num_label),
            # nn.ReLU(),
            # nn.Linear(10000, 20000),
            # nn.ReLU(),
            # nn.Linear(20000, 40000),
            # nn.ReLU(),
            # nn.Linear(40000, num_label),
            # nn.Softmax(1)
        )

    def forward(self, wrapped_input):
        hidden = self.bert(**wrapped_input)
        last_hidden_state, pooler_output = hidden[0], hidden[1]
        logits = self.classifier(pooler_output)
        # logits = self.softmax(logits)

        return logits.squeeze()

    # def forward(self, input_ids, token_type_ids, attention_mask=None):
    #     pooled_out = self.bert(input_ids, token_type_ids, attention_mask)[1]
    #     logits = self.classifier(pooled_out)
    #     return logits

In [ ]:
model = BERT_classifier(NB_CLASSES).to(device)
print(model)

In [ ]:
batch_data, batch_label = next(iter(train_loader))
for k, v in batch_data.items():
    batch_data[k] = v.to(device)
    print(v.shape)
batch_label = batch_label.to(device)

batch_logits = model(batch_data)
print(batch_logits.shape)
print(batch_logits)
print(batch_label)
print(batch_data['input_ids'].shape)
print(type(batch_label[0]))

In [ ]:
# criterion = nn.CrossEntropyLoss(weight=WEIGHT)
criterion = nn.CrossEntropyLoss()
loss = criterion(batch_logits, batch_label)
print(loss)

In [ ]:
def train(model, train_loader, optimizer, criterion, epoch):
    model.train()
    train_loss, correct = 0, 0
    for data, label in tqdm(train_loader):
        device_data = {}
        for k, v in data.items():
            device_data[k] = v.to(device)
        device_label = label.to(device)
        
        optimizer.zero_grad()
        output = model(device_data)
        loss = criterion(output, device_label)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()

        pred = output.argmax(dim=1)
        lab = device_label.argmax(dim=1)
        correct += pred.eq(lab).sum().item()

    train_loss /= len(train_loader)
    acc = correct/len(train_loader.dataset)

    return train_loss, acc


def val(model, val_loader, criterion, min_val_loss):
    model.eval()
    val_loss, correct = 0, 0.
    with torch.no_grad():
        for data, label in val_loader:
            device_data = {}
            for k, v in data.items():
                device_data[k] = v.to(device)
            device_label = label.to(device)
            
            output = model(device_data)
            val_loss += criterion(output, device_label).item()

            pred = output.argmax(dim=1)
            lab = device_label.argmax(dim=1)
            correct += pred.eq(lab).sum().item()
            
        val_loss /= len(val_loader)
        acc = correct/len(val_loader.dataset)
        
    if min_val_loss>val_loss:
        min_val_loss = val_loss
        torch.save(model.state_dict(), 'best-model-parameters.pt')
        print('model saved')

    return val_loss, acc, min_val_loss

def fit(model, train_loader, val_loader, epochs, optimizer, criterion):
    loss_train_per_epoch = []
    acc_train_per_epoch = []
    loss_val_per_epoch = []
    acc_val_per_epoch = []
    min_val_loss = 1000
    for epoch in range(epochs):
        train_loss, train_acc = train(model, train_loader, optimizer, criterion, epoch+1)
        val_loss, val_acc, min_val_loss = val(model, val_loader, criterion, min_val_loss)

        print(f'[{epoch + 1}, {len(train_loader) + 1:5d}] loss: {train_loss:.3f}, accuracy: {train_acc:.3f} loss_val: {val_loss:.3f}, accuracy_val: {val_acc:.3f}')
        
        loss_train_per_epoch += [train_loss]
        acc_train_per_epoch += [train_acc]
        loss_val_per_epoch += [val_loss]
        acc_val_per_epoch += [val_acc]
        
    return loss_train_per_epoch, loss_val_per_epoch, acc_train_per_epoch, acc_val_per_epoch
    
    

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=5e-3)
# criterion = nn.CrossEntropyLoss(weight=WEIGHT)
criterion = nn.CrossEntropyLoss()
epochs=30
loss_train_per_epoch, loss_val_per_epoch, acc_train_per_epoch, acc_val_per_epoch = fit(model, train_loader, val_loader, epochs, optimizer, criterion)

In [ ]:
batch_data, batch_label = next(iter(val_loader))
for k, v in batch_data.items():
    batch_data[k] = v.to(device)
batch_logits = model(batch_data)
print(batch_logits)
print(batch_label)

In [ ]:
def show_loss(loss_train,loss_val):
    plt.figure(figsize=(10,5))
    plt.title("Training and Validation Loss")
    plt.plot(loss_val,label="val")
    plt.plot(loss_train,label="train")
    plt.xlabel("iterations")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()

def show_acuracy(acc_train,acc_val):
    plt.figure(figsize=(10,5))
    plt.title("Training and Validation Accuracy")
    plt.plot(acc_val,label="val")
    plt.plot(acc_train,label="train")
    plt.xlabel("iterations")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.show()

In [ ]:
def show_matrix_confusion(model, val_dataloader):
    model.eval()
    val_acc = 0.0
    true_lab = []
    pred = []
    with torch.no_grad():
        for data, label in val_loader:
            device_data = {}
            for k, v in data.items():
                device_data[k] = v.to(device)
            device_label = label.to(device)
            
            output = model(device_data)
            out = output.argmax(dim=1)
            lab = device_label.argmax(dim=1)
            val_acc += out.eq(lab).sum().item()
            true_lab += [lab.cpu().numpy()]
            pred += [out.cpu().numpy()]
    
    print(f'accuracy: {val_acc / len(val_dataloader.dataset):.3f}')
    y_pred = np.concatenate(pred)
    y_true = np.concatenate(true_lab)

    print("Balanced accuracy :",balanced_accuracy_score(y_true, y_pred))
    
    confusion_matrix = metrics.confusion_matrix(y_true, y_pred)
    cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix)
    
    cm_display.plot()
    plt.show() 

In [ ]:
show_acuracy(acc_train_per_epoch,acc_val_per_epoch)

In [ ]:
show_loss(loss_train_per_epoch,loss_val_per_epoch)

In [ ]:
show_matrix_confusion(model, val_loader)